# Preprocessing text-mining
Ce notebook a pour objectif le traitement des textes obtenus par océrisation des images.
L'objectif est donc d'obtenir, pour chaque image (=document) un texte amélioré, permettant d'alimenter des modèles de Machine learning.

La base de texte choisie est l'ocr collectif: comme cela a été vu procédément, il contient le plus grand nombre de données (environ 398k documents sur les 400k possibles) et les informations contenues semblent identiques à celles des documents individuels).

Dans ce notebook, on va donc créer un dataframe qui aura ce format (les colonnes ocr_x sont des colonnes obtenues après traitement de raw_ocr):

| document_id (index) | raw_ocr | ocr_1 |...| ocr_n | label 

En fin de notebook, il sera ainsi possible de créer les Dataframes X (ocr_x) et y (label) qui permettront d'alimenter les modèles.

La gestion du test_train_split est faite en début de notebook, en respectant la répartition définie dans le dataset RVL-CDIP.

Remarque: des contraintes temporelles (temps alloué au projet + temps de calculs requis) nous ont contraints à nous limiter à un seul jeu de données traitées ("ocr_1"). Nous avons toutefois laissé la structure imaginée pour montrer la démarche dans laquelle nous nous inscrivions.

## 1. Préparation

In [ ]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
import re
import pandas as pd

In [ ]:
df = pd.read_parquet(PATHS.processed_data / "df_raw_ocr.parquet")

# 2. Application des traitements

## 3.1. Création ocr_1
Ce premier pipeline de traitement permettra d'arriver à la création de la colonne ocr_1

### 3.1.1. Suppression des pagesNbr

In [ ]:
pg_regex = re.compile(r'pgNbr=[0-9]+')
type(pg_regex)

In [ ]:
df.head(10)

In [ ]:
def count_pgNbr(text):
    return len(pg_regex.findall(text))
# comptabilisation du nombre de pgNbr:

def remove_pgNbr(text):
    text = pg_regex.sub('', text)
    return text

avant = df.raw_ocr.apply(count_pgNbr).sum()
df["ocr_tmp"] = df.raw_ocr.apply(remove_pgNbr)
apres = df.ocr_tmp.apply(count_pgNbr).sum()
print(avant, apres)

### 3.1.2. Déséchappement html

In [ ]:
import html
def unescape_html(text):
    if not text:
        return text
    return html.unescape(text)

avant = df.ocr_tmp.str.contains("&lt;").sum()
df.loc[:,"ocr_tmp"] = df.ocr_tmp.apply(unescape_html)
apres = df.ocr_tmp.str.contains("&lt;").sum()
avant, apres

### 3.1.3. Correction OCR
Nous allons pour cela utiliser l'outil jamspell

In [ ]:
# Le but ici est d'utiliser une librairie de correction de texte océrisé nommée jamspell
# L'emploi de cette bibliothèque peut se faire soit avant, soit après les traitements réalisés au 1.
# Nous utiliserons les 2 approches, pour réaliser 2 colonnes:
# - cleaned_ocr_jamspell_first 
# - cleaned_ocr_jamspell_last

In [ ]:
import jamspell
jamspell_model_path = os.path.join(project_path, 'models', 'jamspell', 'en.bin')
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel(jamspell_model_path)
def apply_jamspell(text):
    if not text:
        return text
    return corrector.FixFragment(text)
# avant = df.ocr_tmp.str.contains("&lt;").sum()
df.loc[:,"ocr_tmp"] = df.ocr_tmp.apply(apply_jamspell)
# apres = df.ocr_tmp.str.contains("&lt;").sum()
# avant, apres

In [ ]:
df.head(20)

### 3.1.4 Suppression des caractères spéciaux et des séquences ne correspondant pas à des informations

In [ ]:
import re

# A améliorer pour prendre en compte les lignes
# sans doute trop brutal / il faudra le réviser sur d'autres versions ultérieures?
def basic_word_filter(text):
    if not text:
        return text
    text = text.lower()
    # Attention, c'est brutal, ca supprime tous les chiffres aussi...
    word_regex = re.compile(r'[a-z]{2,}')
    text = ' '.join(word_regex.findall(text))
    
    return text
df["ocr_tmp"] = df.ocr_tmp.apply(basic_word_filter)

### 3.1.5. Filtrage des stop_words

In [ ]:

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
nltk.download('punkt_tab')
# Télécharger les ressources NLTK nécessaires
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
#Liste des stop words en anglais
stop_words = set(stopwords.words('english'))

# Fonction pour nettoyer une phrase
def remove_stopwords(text):
    if pd.isnull(text):  # gestion des valeurs manquantes
        return ""
    words = word_tokenize(text.lower())
    filtered = [word for word in words if word.isalpha() and word not in stop_words]
    return " ".join(filtered)

# Application sur la colonne raw_ocr
df['raw_ocr_clean'] = df['ocr_tmp'].apply(remove_stopwords)

# Affichage
print(df[['ocr_tmp', 'raw_ocr_clean']])


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Pour affichage dans le notebook (spécifique Jupyter sur macOS)
%matplotlib inline

# Préparer le texte (limité à 500 000 caractères pour éviter les crashs)
text = " ".join(df['ocr_tmp'].dropna().astype(str).tolist())[:500000]

# Générer le nuage sans stopwords
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Affichage
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Nuage de mots (sans suppression)")
plt.show()




In [ ]:
df.head()

### 3.1.6. Application du Pipeline et création d'ocr_1

In [ ]:
# Todo
def clean_ocr_1(text):
    text = remove_page_number(text)
    text = unescape_html(text)
    text = apply_jamspell(text)
    text = basic_word_filter(text)
    text = remove_stopwords(text)
    
    # ...
    return text
if "ocr_1" in train.columns:
    train.drop(columns="ocr_1", inplace=True)
train.insert(
    loc=1,
    column="ocr_1",
    value=train["raw_ocr"].apply(clean_ocr_1),
    allow_duplicates=False
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

#Séparer les jeux
train_df = df[df['data_set'] == 'train']
val_df   = df[df['data_set'] == 'val']
test_df  = df[df['data_set'] == 'test']  # facultatif pour l’instant

# 3. Features et labels
X_train = train_df['ocr_tmp']
y_train = train_df['label']

X_val = val_df['ocr_tmp']
y_val = val_df['label']

# 4. Vectorisation TF-IDF sur le train uniquement
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vect = vectorizer.fit_transform(X_train)
X_val_vect   = vectorizer.transform(X_val)


In [ ]:
 #Modèles à tester
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "Naive Bayes": MultinomialNB()}


### 3.1.7. Analyse des résultats obtenus

In [ ]:
#Entraînement et évaluation
for name, model in models.items():
    model.fit(X_train_vect, y_train)
    y_pred = model.predict(X_val_vect)
    print(f"\n--- {name} ---")
    print(classification_report(y_val, y_pred))

In [ ]:
from sklearn.metrics import classification_report

# Dictionnaire pour stocker les métriques
scores = {}

for name, model in models.items():
    model.fit(X_train_vect, y_train)
    y_pred = model.predict(X_val_vect)
    print(f"\n--- {name} ---")
    print(classification_report(y_val, y_pred))

    # Stockage des métriques
    report = classification_report(y_val, y_pred, output_dict=True)
    scores[name] = {
        'precision': report['macro avg']['precision'],
        'recall': report['macro avg']['recall'],
        'f1-score': report['macro avg']['f1-score']
    }




In [ ]:
import matplotlib.pyplot as plt

# Extraire les métriques
labels = list(scores.keys())
precision = [scores[model]['precision'] for model in labels]
recall = [scores[model]['recall'] for model in labels]
f1 = [scores[model]['f1-score'] for model in labels]

x = range(len(labels))
width = 0.25

# Graphique
plt.figure(figsize=(10, 6))
plt.bar([p - width for p in x], precision, width=width, label='Précision')
plt.bar(x, recall, width=width, label='Rappel')
plt.bar([p + width for p in x], f1, width=width, label='F1-score')

plt.xticks(x, labels, rotation=45)
plt.ylabel("Score (macro avg)")
plt.title("Comparaison des métriques par modèle")
plt.ylim(0, 1.05)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
!pip install tensorflow-macos)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Encodage des labels (si ce n’est pas encore fait)
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_val_enc = encoder.transform(y_val)

# Définition du modèle MLP
model_mlp = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_vect.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(set(y_train_enc)), activation='softmax')  # Softmax pour multi-classe
])

model_mlp.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Entraînement
model_mlp.fit(X_train_vect.toarray(), y_train_enc,
              epochs=5, batch_size=32,
              validation_data=(X_val_vect.toarray(), y_val_enc))


In [ ]:

# Créer le chemin vers Documents
import os

home = os.path.expanduser("~")
documents_path = os.path.join(home, "Documents")

# Sauvegarder les fichiers dans Documents
train_df.to_csv(os.path.join(documents_path, "train_df.csv"), index=False)
test_df.to_csv(os.path.join(documents_path, "test_df.csv"), index=False)
val_df.to_csv(os.path.join(documents_path, "val_df.csv"), index=False)

print("✅ Fichiers enregistrés dans ton dossier Documents !")
